A base de dados escolhida consiste em um conjunto de dados que descrevem a rotina de compra de um ecommerce. Nela podemos visualizar diversas características de um produto, como o nome, preço, descrição, nota atribuída, comentários, local de compra, etc. 

Por ser tão abrangente, essa análise será limitada ao sistema de avaliação feito pelos clientes após a confirmação de entrega do produto. Assim, o consumidor pode dar nota de 1 a 5 para o produto, sendo 1 o mais baixo e 5 o mais alto. Os comentários serão desconsiderados para essa primeira proposta.

Com essas avaliações, a plataforma de e-commerce consegue tirar insights sobre a qualidade do produto. Com essas avaliações, consegue-se trazer melhorias para o negócio. A grande questão que surge é: Seria possível prever a pontuação dos clientes com base em alguma outra métrica antes mesmo dessa avaliação ser efetuada? Caso sim, também seria possível prever os casos sem avaliação. No estudo em questão, o principal objetivo é resolver esse problema, que é a previsão da satisfação do cliente para que o comércio eletrônico possa lançar mão de meios para mitigar eventuais danos em determinados casos.

Alguns requisitos que devem ser observados são:

1) Classificações baixas (1,2 e 3) são essenciais para a melhoria do negocio e devem ser analisada e classificadas corretamente;
2) O modelo não deve demorar muito para ser executado e fazer as previsões, já que essa previsão deve ser feita antes que o usuário dê a nota.